In [10]:
from HyperparamCalculator import HyperparamCalculation, predictByModel
import pandas as pd
import numpy as np

titanic_data = pd.read_csv("data/titanic_prepared.csv").drop(['Unnamed: 0'], axis=1)

titanic_full_df = titanic_data.sample(frac=1, random_state=5)
testPart = int(len(titanic_full_df)*0.1)
test_df = titanic_full_df[:testPart]
train_df = titanic_full_df[testPart:]

def divideData(df):
    return (df.drop(['label'], axis=1), df.label)

test_x, test_y = divideData(test_df)
train_x, train_y = divideData(train_df)

## Decision Tree

In [11]:
from sklearn.tree import DecisionTreeClassifier

paramsDecTree = {
    "max_depth": list(range(1, 15)),
    "criterion": ("gini", "entropy", "log_loss")
}

decTreeParametrizer = lambda p: DecisionTreeClassifier(max_depth=p[0], criterion=p[1])
decTreeCalculator = HyperparamCalculation(trainData=(train_x, train_y), validData=(test_x, test_y), setParamFunc=decTreeParametrizer, iterationsCnt=50)
DecTreeParams = decTreeCalculator.calculate(paramsDecTree, testIterationsCnt=30)

modelDecTree = decTreeParametrizer(DecTreeParams)
modelDecTree.fit(train_x, train_y)
result = predictByModel(modelDecTree, test_x, test_y, modelName="DecisionTree", dataType='Test')

Лучшая точность (Validation): 0.9064856711915537
Лучшие параметры (Validation): (6, 'entropy')
Точность определения (Test): 90.65% (DecisionTree)
